In [1]:
!pip3 install xlsxwriter

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import xlsxwriter
from xlsxwriter.utility import xl_rowcol_to_cell
import os, glob
import re
import numpy as np

path = './Conversation history'


def addItems(worksheet, columns):
    worksheet.write('A1', 'Filename')
    worksheet.write('B1', 'Condition')
    worksheet.write('C1', 'Path')
    for col in np.arange(len(columns)):
        worksheet.write(0, 3+col, columns[col])
    worksheet.write(0, 3+col+1, 'Percentage')
    worksheet.write(0, 3+col+2, 'Completion')
    worksheet.write(0, 3+col+3, 'Error')
    left = 'D2'
    right = xl_rowcol_to_cell(1, 3+col)
    bottom = xl_rowcol_to_cell(100, 3+col)
    ## write the percentage calculation formula at (1, 3+col+1)
    worksheet.write_formula(1, 3+col+1, "=Average("+left+":"+ right+")")
    worksheet.data_validation('D2:{}'.format(bottom),{'validate': 'list',
                                                      'source': [0, 1]})
    top_completion = xl_rowcol_to_cell(1, 3+col+2)
    bottom_completion = xl_rowcol_to_cell(100, 3+col+2)
    worksheet.data_validation('{}:{}'.format(top_completion, bottom_completion),{'validate': 'list',
                                                      'source': [0, 1]})
    top_error = xl_rowcol_to_cell(1, 3+col+3)
    bottom_error = xl_rowcol_to_cell(100, 3+col+3)
    worksheet.data_validation('{}:{}'.format(top_error, bottom_error),{'validate': 'list',
                                                      'source': ['None', 'Repetition', 'Loop', 'Self-talk', 'Others']})
    return worksheet



columns_sleep = ['Time to bed', 'Time took to fall asleep', 'Wake up at night', 'Wake up time', 'Sleep quality (1 to 10)']
columns_work = ['Work done', 'Rate productivity (1 to 10)', 'Other concerns at work', 'What went well']
columns_exercise = ['What workout', 'Workout duration', 'Feeling after workout', 'Fitness concerns']
columns_diet = ['Breakfast', 'Lunch', 'Dinner', 'Snacks', 'Feelings after eating']

workbook = xlsxwriter.Workbook('rater_file.xlsx')
worksheet_sleep = addItems(workbook.add_worksheet('Sleep'), columns_sleep) 
worksheet_work = addItems(workbook.add_worksheet('Work'), columns_work)
worksheet_exercise = addItems(workbook.add_worksheet('Exercise'), columns_exercise)
worksheet_diet = addItems(workbook.add_worksheet('Diet'), columns_diet)

c_sleep = 1
c_work = 1
c_exercise = 1
c_diet = 1

## below is the excel file formatting
glob.glob(os.path.join(path, '*.txt'))
for filename in glob.glob(os.path.join(path, '*/*/*.txt')):
    read_filename = re.split('/',filename)[-1]
    read_info = re.split('_|\.', read_filename)
    condition = read_info[1]
    topic = read_info[0]
    path = read_info[2]
    if topic == 'sleep':
        worksheet_sleep.write_url(c_sleep, 0, r'external:'+filename)
        worksheet_sleep.write(c_sleep, 1, condition)
        worksheet_sleep.write(c_sleep, 2, path)
        c_sleep += 1
    elif topic == 'work':
        worksheet_work.write_url(c_work, 0, r'external:'+filename)
        worksheet_work.write(c_work, 1, condition)
        worksheet_work.write(c_work, 2, path)
        c_work += 1
    elif topic == 'exercise':
        worksheet_exercise.write_url(c_exercise, 0, r'external:'+filename)
        worksheet_exercise.write(c_exercise, 1, condition)
        worksheet_exercise.write(c_exercise, 2, path)
        c_exercise += 1
    else:
        worksheet_diet.write_url(c_diet, 0, r'external:'+filename)
        worksheet_diet.write(c_diet, 1, condition)
        worksheet_diet.write(c_diet, 2, path)
        c_diet += 1

        
workbook.close()


In [13]:
import numpy as np
columns_work = ['Work done', 'What went well', 'Rate productivity (1 to 10)', 'Other concerns at work']
for i in np.arange(4):
    print(columns_work[i])
i

Work done
What went well
Rate productivity (1 to 10)
Other concerns at work


3

In [50]:
import xlsxwriter
from xlsxwriter.utility import xl_rowcol_to_cell
import os, glob
import re
import numpy as np
import pandas as pd

path = './Conversation history'
dd = pd.read_csv('participant_id.csv')
conv_all = []
for filename in glob.glob(os.path.join(path, '*/*/*.txt')):
    read_filename = re.split('/',filename)[-1]
#     print(filename)
    email_id = re.split('/',filename)[2]
    read_info = re.split('_|\.', read_filename)
    condition = read_info[1]
    topic = read_info[0]
    path = read_info[2]
    participant = dd[dd['Email'] == email_id]['Alias'].values[0]
    with open(filename, 'r') as f:
        content = f.read()
        dialog_num = content.find('[Dialog]') 
        dialog = content[dialog_num+9:]
        # remove one outlier
        dialog = dialog.replace('\n\n', ' ')
        # convert string to a list
        dialog_ls = dialog.split('\n')
        

        if int(condition) > 4:
            condition = str(int(condition) - 4)

        conv_cur = []
        turn_num = 0
        for turn in dialog_ls:
            if turn[0] == 's':
                speaker = 'system'
                turn_num += 1
            elif turn[0] == 'u':
                speaker = 'user'
                turn_num += 1
            else:
                speaker = 'ERROR'
            conv_cur.append([participant, condition, topic, path, turn_num, turn, speaker])
            
        conv_all += conv_cur

        
df = pd.DataFrame(data=conv_all, columns = ['Participant', 'Condition', 'Topic', 'Path', 'Turn', 'Utterance', 'Speaker'])


In [51]:
df.to_excel('conversation_data.xlsx')

In [49]:
df['Participant'][0][0]

'C'

In [42]:
email_id

'kwh062793@naver.com'